# Multibyte Addition

- Multibyte arithmetic is done on values stored in memory. Each byte is moved into the accumulator as it is needed.
<br><br>
- In multibyte arithmetic, we usually don't work with values that have been converted to pure binary form. We instead work with a data representation system called binary coded decimal (BCD). This is also referred to as packed decimal.

<img src="images/bcd-hex.png">

- We see that in the hex value of equivalent BCD is equal to the decimal value.
<br><br>
- In BCD a byte is split into halves (called nibbles) and both nibbles represent a number between 0 to 9. Values A to F are forbidden. E.g:- 83 when converted to BCD has 8 in the upper nibble and 3 in the lower one.
<br><br>
- To prevent the resulting BCD from being invalid, we use an instruction called SED (SEt Decimal mode). This has no operands, and instructs the processor to use BCD arithmetic until further notice.

## Questions

1) Write a set of instructions that will move the least significant byte of ADDEND1 into the accumulator, then add the least significant byte of ADDEND2 to it using BCD arithmetic. Put the result in the LSByte of ADEND2:-

```assembly
SED
LDX  #1
LDA  ADEND1,X
CLC
ADC  ADEND2,X
STA  ADEND2,X
```

2) Write a set of instruction that will move the ADEND1 byte into the accumulator, add the ADEND2 byte to it, plus the value in the carry flag, store it in the correct byte of ADEND2 (this adds 2 bytes):-

```assembly
ADDER    EQU  *
         SED
         LDX  #1
         CLC
         LDA  ADEND1,X
         ADC  ADEND2,X
         STA  ADEND2,X
         DEX
         LDA  ADEND1,X
         ADC  ADEND2,X
         STA  ADEND2,X
```

3) Adapt the routine in question 2 so that it adds two five-byte numbers and stores the sum:-

```assembly
ADDER    EQU  *
         SED
         LDX  #4
         CLC
ADDBYT   EQU  *
         LDA  ADEND1,X
         ADC  ADEND2,X
         STA  ADEND2,X
         DEX
         BPL  ADDBYT
```

# BCD Conversion

## Questions

1) Read one digit from the terminal and convert it from ASCII to binary. All you have to do is turn off all the bits in the MSN (MS Nibble), leave the result in the accumulator:-

```assembly
JSR  INPUT
JSR  OUPUT
AND  #%00001111
```

2) In BCD it's very important that only valid decimal digits are used. Convert the routine you coded in the previous question into a complete subroutine that gets one valid digit:-
<br><br>
    - Read and echo one character.
<br><br>
    - Validate range '0' to '9' (make sure the character is in the range).
<br><br>
    - If the character is out of range, write an error message and try again.
<br><br>
    - When a valid digit is obtained, convert it to binary and return control. Leave the new value in A.
    
```assembly
GETDIG    EQU  *
          STX  HOLDX
AGAIN     EQU  *
          JSR  INPUT
          JSR  OUTPUT
          CMP  #'0'
          BMI  ERROR
          CMP  #':'      ; COMPARE TO CHARACTER AFTER '9'
          BPL  ERROR
          AND  #%00001111
          LDX  HOLDX
          RTS
ERROR     EQU  *
          LDX  #0
MSGOUT    EQU  *
          LDA  ERRMSG,X
          JSR  OUTPUT
          INX
          CPX  #26
          BNE  MSGOUT
          JMP  AGAIN
HOLDX     DS   1
ERRMSG    ASC  'INVALID DIGIT -- TRY AGAIN'
```

- To extend this for two digits we can take the following steps:-
<br><br>
    - Rotate the first digit into the upper nibble
<br><br>
    - Add in the second digit
- Code a routine that will get two valid decimal digits (by calling GETDIG) and create once BCD byte. Store the byte in ADEND1:-

```assembly
GETNUM    EQU  *
          JSR  GETDIG  ; GET MSBYTE
          ASL  A
          ASL  A
          ASL  A
          ASL  A
          STA  MSB
          JSR  GETDIG  ; GET LSBYTE
          CLC
          ADC  MSB
          STA  ADDEND1
MSB       DS   1
```

3) Complete the routine so that it gets and stores all of ADEND1 (ten digits = five bytes) and ADEND2 (also five bytes). For the user's benefit, start a new line on the display screen after each 10-digit number. You'll want to use a 'nested loop'. Have an outer loop that executes the inner loop twice for the two numbers:-

```assembly
CR        EQU  $0D
LF        EQU  $0A
GETADS    EQU  *
          LDX  #0       ; INITIALIZE INDEX
          LDA  #2       ; CONTROL OUTER LOOP FOR 2 NUMBERS
          STA  COUNT2
GETAD     EQU  *
          LDA  #5       ; CONTROL INNER LOOP FOR 5 BYTES PER NUMBER
          STA  COUNT5
GETNUM    EQU  *
          JSR  GETDIG
          ASL  A
          ASL  A
          ASL  A
          ASL  A
          STA  MSB
          JSR  GETDIG
          CLC
          ADC  MSB
          STA  ADEND1,X  ; IF X > 4, WE'RE IN ADEND2
          INX
          DEC  COUNT5    ; COUNT INNER LOOP
          BNE  GETNUM
          LDA  #CR
          JSR  OUTPUT
          LDA  #LF       
          JSR  OUTPUT
          DEC  COUNT2
          BNE  GETAD     ; GET THE SECOND ADDEND
COUNT2    DS   1
COUNT5    DS   1
ADEND1    DS   5
ADEND2    DS   5
MSB       DS   1
```

4) Code a routine that will:-
<br><br>
    - Get one byte from ADEND2.
<br><br>
    - Split the two nibbles.
<br><br>
    - Convert to ASCII.
<br><br>
    - Write both digits.
<br><br>
    - Repeats (1) to (4) until the entire sum is written out.
    
```assembly
MASCII    EQU  %00110000
WRITIT    EQU  *
          LDX  #0
CONVRT    EQU  *
          LDA  ADEND2,X       ; GET BYTE
          LSR  A
          LSR  A
          LSR  A
          LSR  A
          ORA  #MASCII        ; CONVERT TO ASCII
          JSR  OUTPUT
          LDA  ADEND2,X       ; GET IT AGAIN
          AND  #%00001111     ; USE LOWER NIBBLE
          ORA  #MASCII        ; CONVERT TO ASCII
          JSR  OUTPUT
          INX
          CPX  #5
          BNE  CONVRT
```

- The following program shows the entire program to read and add two ten-digit numbers. This program has some weak points:-
<br><br>
    - The user must type all 10 digits including leading zeros.
<br><br>
    - There aren't any steps to prevent overflow of the sum.
<br><br>
    - The errror message routine dumps the message in the middle of the user's number.
    
```assembly
MASCII    EQU  %00110000
CR        EQU  $0D
LD        EQU  $0A
GETADS    EQU  *
          LDX  #0  ; INITIALIZE INDEX
          LDA  #2  ; CONTROL OUTER LOOP
GETAD     EQU  *
          LDA  #5  ; CONTROL INNER LOOP
          STA  COUNT5
GETNUM    EQU  *
          JSR  GETDIG
          ASL  A
          ASL  A
          ASL  A
          ASL  A
          STA  MSB
          JSR  GETDIG   ; GET LSB
          CLC
          ADC  MSB
          STA  ADEND1,X    ; IF X > 4 WE'RE IN ADEND2
          INX
          DEC  COUNT5
          BNE  GETNUM
          LDA  #CR
          JSR  OUTPUT
          LDA  #LF
          JSR  OUTPUT
          DEC  COUNT2
          BNE  GETAD
ADDER     EQU  *
          SED
          LDX  #4
          CLC
ADDBYT    EQU  *
          LDA  ADEND1,X
          ADC  ADEND2,X
          STA  ADEND2,X
          DEX
          BPL  ADDBYT
WRITIT    EQU  *
          LDX  #0
CONVRT    EQU  *
          LDA  ADEND2,X
          AND  #%111100000 ; USE UPPER NIBBLE
          LSR  A
          LSR  A
          LSR  A
          LSR  A
          ORA  #MASCII
          JSR  OUTPUT
          LDA  ADEND2,X
          AND  #%00001111 ; USE LOWER NIBBLE
          ORA  #MASCII
          JSR  OUTPUT
          INX
          CPX  #5
          BNE  CONVRT
DONE      JMP  DONE
GETDIG    EQU  *
          STX  HOLDX
AGAIN     EQU  *
          JSR  INPUT
          JSR  OUTPUT
          CMP  #'0'
          BMI  ERROR
          CMP  #':'
          BPL  ERROR
          AND  #%00001111
          LDX  HOLDX
          RTS
ERROR     EQU  *
          LDX  #0
MSGOUT    EQU  *
          LDA  ERRMSG,X
          JSR  OUTPUT
          INX
          CPX  #26
          BNE  MSGOUT
          JMP  AGAIN
COUNT2    DS   1
COUNT5    DS   1
ADEND1    DS   5
ADEND2    DS   5
MSB       DS   1
HOLDX     DS   1
ERRMSG    ASC  'INVALID DIGIT -- TRY AGAIN'
```

# Multiplication

- Multiplcation is basically repetitive addition.
<br><br>
- There are no seperate multiplication instructions. We can multiply using, left shift. Each shift multiplies the accumulator by two.

## Questions

1) The first shift left multiplies by two. The second doubles the first, or multiplies by four. The third doubles the second or multiplies by 8. Using ASL, write a set of instructions to multiply by 16:-

```assembly
ASL A  ; TIMES 2
ASL A  ; TIMES 4
ASL A  ; TIMES 8
ASL A  ; TIMES 16
```

2) Write a set of instructions to read an ASCII byte, convert it to binary, multiply it by 8, and store it at BYTEX8:-

```assembly
JSR INPUT
JSR OUTPUT
AND #%00001111
ASL A ; TIMES 2
ASL A ; TIMES 4
ASL 8 ; TIMES 8
STA BYTEX8
```

## Multiplication for numbers that are not powers of 2

- We can accomplish any multiplicand by adding together the various powers of two. For example, to multiply by 5:-
<br><br>
    - Put the original number in HOLDA.
<br><br>
    - Shift left once (2A).
<br><br>
    - Shift left again (4A).
<br><br>
    - Add the original byte from HOLDA (4A + A = 5A).
    
## Questions

1) Write a routine to multiply the value in accumulator by 7:-

```assembly
STA HOLDA
ASL A     ; TIMES 2
CLC
ADC HOLDA ; TIMES 3
ASL A     ; TIMES 6
ADC HOLDA ; TIMES 7
```

2) Write a routine to multiply the value in accumulator by 10:-

```assembly
STA HOLDA
ASL A     ; TIMES 2
ASL A     ; TIMES 4
ADC HOLDA ; TIMES 5
ASL A     ; TIMES 10
```

## BCD Multiplication

- The following routine multiplies by 10:-

```assembly
        SED
        LDX  #10
        STA  HOLDA
MULTY   EQU  *
        DEX
        BEQ  DONE
        CLC
        ADC  HOLDA
        JMP  MULTY
DONE    EQU  *
```

## Multibyte BCD Multiplication

- Suppose we have to multiply a multibyte BCD value, this three-byte BCD value is called MULTER:-

<img src="images/multer.png">

- We want to multiply it by six, and store the product in MULTED, which has 4 bytes and an initial value of 0:-

<img src="images/multed.png">

- This can be done by a loop that adds MULTER to MULTED six times:-

```assembly
        SED
        LDA  #6      ; LOOP
        STA  COUNT6  ; COUNTER
ONEADD  EQU  *
        LDX  #2      ; INDEX FOR MULTER
        LDY  #3      ; INDEX FOR MULTED
        CLC
NEXBYT  EQU  *
        LDA  MULTED, Y
        ADC  MULTER, X
        STA  MULTED, Y
        DEY
        DEX
        BPL  NEXBYT
        LDA  MULTED, Y
        ADC  #0      ; ADD CARRY TO MSB OF MULTED
        STA  MULTED, Y
        DEC  COUNT6
        BNE  ONEADD
```

# Division

- Division is a process of repeated subtraction.
<br><br>
- The following routine divides by two, we assume that the dividend is already in the accumulator, the quotient will be stored in X:-

```assembly
        LDX  #0     ; CLEAR THE QUOTIENT
SUBIT   EQU  *
        CMP  #2     ; CAN WE MAKE ANOTHER SUBTRACTION?
        BMI  DONE
        SEC
        SBC  #2     ; SUBTRACT DIVISOR FROM A
        INX         ; COUNT SUBTRACTION
        JMP  SUBMIT
DONE    EQU  *
```

- The quotient will end up in X and the remainder in A.

## Questions

1) Change the above routine to divide any number between 1 and 255. Assume the divisor is in DIVISR:-

```assembly
        LDX  #0      ; CLEAR THE QUOTIENT
SUBIT   EQU  *
        CMP  DIVISR  ; CAN WE SUBTRACT?
        BMI  DONE
        SEC
        SBC  DIVSR   ; SUBTRACT DIVISOR FROM A
        INX
        JMP  SUBIT
DONE    EQU  *
```

- Here 0 in DIVISR can cause a closed loop.

# Handling Negative Numbers

- Negative numbers are stored using twos complement notation.
<br><br>
- When working with 8-bit numbers, 2s complements are two numbers that add up to %100000000. When we add complementary numbers in the accumulator, the result is zero with the carry flag on.
<br><br>
- In binary an easy way to find 2s complement is to flip every bit and add 1 to the resulting number. An example is:-

<img src="images/twos-comp-bin.png">

- So converting all negative numbers to 2s complement as soon as they are entered and then let all additions and subtractions proceed as if only positive numbers were in use, except ignoring the carry flag is thus the simplest solution to handle negative numbers.
<br><br>
- The following routine computes 2s complement in 6502:-

```assembly
LDA NEGNUM
EOR #%11111111 ; TO FLIP THE BITS
CLC
ADC #1
```

## Questions

1) Write a set of instructions to find the 2s complement of SUBBER. Leave the result in SUBBER:-

```assembly
LDA SUBBER
EOR #%11111111 ; TO FLIP THE BITS
CLC
ADC #1
STA SUBBER
```

## Continuation

- When we are using a 2s complement system, we let the MSB act as sign indicator. If it's on, the value is negative, otherwise the value is positive. So, we have to limit positive values to %01111111 per byte.
<br><br>
- While converting to decimal if the MSB is off then convert as is, otherwise take 2s complement of the number and then apply negative sign before it (to denote negative number).
<br><br>
- In 2s complement system, the maximum positive value per byte is 127 and maximum negative value that will fit in a byte is -128.

## Adding two single digits 

- The following routine gets one byte and if its negative then converts it to 2s complement:-

```assembly
GETBYT  EQU  *
        JSR  INPUT
        JSR  OUTPUT
        CMP  #'-'       ; IS IT A MINUS SIGN?
        BEQ  NEGIVE
        AND  #%00001111 ; CONVERT TO BINARY
        JMP  ENDING
NEGIVE  EQU  *
        JSR  INPUT
        JSR  OUTPUT
        AND  #%00001111 ; CONVERT TO BINARY
        EOR  #%11111111 ; FLIP BITS
        CLC
        ADC  #1         ; 2S COMPLEMENT
ENDING  EQU  *
        RTS
```

- The following routine gets two bytes, stores the first byte in memory, leaves the second byte in A. Add the two bytes and leave the sum in A:-

```assembly
ADDER   EQU  *
        JSR  GETBYT
        STA  ADDEND
        JSR  GETBYT
        CLC
        ADC  ADDEND
```

- The following routine (SUMPOS) checks the size of result. If it's under ten, converts the value in A to ASCII and writes it out, otherwise branches to a routine named TWODIG:-

```assembly
SUMPOS  EQU  *
        CMP  #10
        BCS  TWODIG
        ORA  #%00110000  ; CONVERT TO ASCII
        JSR  OUTPUT
```

- The following routine (TWODIG) converts the result to two decimal digit if sum is positive and larger than nine. Since BCD isn't used, it can be done as:-
<br><br>
    - Divide the value in A by 10.
<br><br>
    - The quoutient is the most significant digit, convert it to ASCII and write it out.
<br><br>
    - The remainder is the least significant digit, convert it to ASCII and write it out.

```assembly
TWODIG  EQU  *
        LDX  #0          ; X WILL HOLD QUOTIENT (HIGH ORDER DIGIT)
DIVIDE  EQU  *
        SEC
        SBC  #10
        INX
        CMP  #10         ; ARE THERE ANY 10'S LEFT?
        BCS  DIVIDE      ; BRANCH IF 10 OR GREATER
        TAY              ; TEMP HOLD REMAINDER (LOW ORDER DIGIT)
        TXA              ; PUT QUOTIENT INTO A
        ORA  #%00110000  ; CONVERT TO ASCII
        JSR  OUTPUT
        TYA              ; PUT REMAINDER INTO A
        ORA  #%00110000  ; CONVERT TO ASCII
        JSR  OUTPUT
```